<a href="https://colab.research.google.com/github/bijmuj/SketchColorization/blob/master/sketch_colorization_AE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Jun 18 14:45:26 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tensorflow-addons
!pip install wandb
!pip install kaggle
!mkdir ~/.kaggle
!chmod 600 kaggle.json
!cp kaggle.json ~/.kaggle
!kaggle datasets download -d ktaebum/anime-sketch-colorization-pair
!unzip -q /content/anime-sketch-colorization-pair.zip -x "data/data/*" "data/colorgram/*"

     |████████████████████████████████| 686kB 7.4MB/s 
     |████████████████████████████████| 1.8MB 7.4MB/s 
     |████████████████████████████████| 133kB 52.7MB/s 
     |████████████████████████████████| 102kB 14.9MB/s 
     |████████████████████████████████| 174kB 48.9MB/s 
     |████████████████████████████████| 71kB 11.4MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8807 sha256=6dd62a46c10c832d813af8778fcb6b1f484ca94367e14c7cd00c2310da63ea11
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6502 sha256=de3f4f2ca88ab3bc473e504e165c09a78dfc3fe4e46f45b6433e1130433dbf5e
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32
100% 11.6G/11.6G [04:41<00:00, 58.0MB/s]
100% 11.6G/11.6G [04:41<00:00, 44.4MB/s]


In [ ]:
import os
import glob
import time
import wandb
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input,Conv2D, Conv2DTranspose, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.models import Model
from tqdm import tqdm
from PIL import Image as im

In [ ]:
BATCH_SIZE = 28
BUFFER_SIZE = 800
EPOCHS = 20
LR = 1e-4
beta_1 = 0.5
beta_2 = 0.999
initializer = tf.random_normal_initializer(0, 0.02)

In [ ]:
wandb.login()
run = wandb.init(project="sketch_colorization", entity="bijin")
config = wandb.config
config.opt = "adam"
config.lr = LR
config.beta_1 = 0.5
config.beta_2 = 0.999
config.batch_size = 16

wandb: Currently logged in as: bijin (use `wandb login --relogin` to force relogin)


In [ ]:
n_batches = 14224/BATCH_SIZE
n_batches

508.0

In [ ]:
def load_image(path):
    image = tf.io.read_file(path)
    image = tf.io.decode_png(image, channels=3)
    # cast and normalize
    image = tf.cast(image, tf.float32)/127.5 - 1
    w = tf.shape(image)[1]
    w = w // 2
    # split
    label = image[:, :w, :]
    sketch = image[:, w:, :]
    # resize
    # not doing this causes issues down the road
    label = tf.image.resize(label, (256, 256))
    sketch = tf.image.resize(sketch, (256, 256))
    return label, sketch

In [ ]:
dataset_path = glob.glob('/content/data/train/*.png')
dataset = tf.data.Dataset.from_tensor_slices(dataset_path)

In [ ]:
dataset = dataset.map(load_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
dataset

<ParallelMapDataset shapes: ((256, 256, 3), (256, 256, 3)), types: (tf.float32, tf.float32)>

In [ ]:
train_dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
dataset_path = glob.glob('/content/data/val/*.png')
val_dataset = tf.data.Dataset.from_tensor_slices(dataset_path)
val_dataset = val_dataset.map(load_image, num_parallel_calls = tf.data.experimental.AUTOTUNE)
val_dataset = val_dataset.batch(5)

In [ ]:
def encoder_unit(input, filters, norm=None, act=None):
    if norm=="Instance":
        x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True, 
                                             beta_initializer="random_uniform",
                                             gamma_initializer="random_uniform")(input)
    elif norm =="Batch":
        x = BatchNormalization()(input)
    else:
        x = input
    x = Conv2D(filters, kernel_size=(4, 4), strides=(2, 2), padding="same", kernel_initializer=initializer)(x)
    if act=="LReLU":
        x = LeakyReLU(0.2)(x)
    return x

In [ ]:
def decoder_unit(input, filters, norm=None, act=None):
    if norm=="Instance":
        x = tfa.layers.InstanceNormalization(axis=3, center=True, scale=True,
                                             beta_initializer="random_uniform",
                                             gamma_initializer="random_uniform")(input)
    elif norm=="Batch":
        x = BatchNormalization()(input)
    else:
        x = input
    x = Conv2DTranspose(filters, kernel_size=4, strides=(2, 2), padding='same', kernel_initializer=initializer)(x)
    if act=="LReLU":
        x = LeakyReLU(0.2)(x)
    elif act=="tanh":
        x = tf.math.tanh(x)
    return x 

In [ ]:
def autoencoder():
    input = Input((256, 256, 3))
    x = encoder_unit(input, 32, act="LReLU")
    x = encoder_unit(x, 64 , norm="Instance", act="LReLU")
    x = encoder_unit(x, 128, norm="Instance", act="LReLU")
    x = encoder_unit(x, 256, norm="Instance", act="LReLU")
    x = encoder_unit(x, 512, norm="Instance", act="LReLU")

    x = decoder_unit(x, 256, norm="Instance", act="LReLU")
    x = decoder_unit(x, 128, norm="Instance", act="LReLU")
    x = decoder_unit(x, 64, norm="Instance", act="LReLU")
    x = decoder_unit(x, 32, norm="Instance", act="LReLU")
    x = decoder_unit(x, 3, norm="Instance", act="tanh")
    return Model(inputs=[input], outputs=x)

In [ ]:
opt = tf.keras.optimizers.Adam(LR, beta_1=beta_1, beta_2=beta_2)
l1_loss = tf.keras.losses.MeanAbsoluteError()
ae = autoencoder()
ae.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 128, 128, 32)      1568      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 128, 128, 32)      0         
_________________________________________________________________
instance_normalization (Inst (None, 128, 128, 32)      64        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        32832     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 64, 64, 64)        0         
_________________________________________________________________
instance_normalization_1 (In (None, 64, 64, 64)        128   

In [ ]:
ckpt_dir = '/content/ckpt'
ckpt = tf.train.Checkpoint(ae=ae,
                           opt=opt)
manager = tf.train.CheckpointManager(ckpt, ckpt_dir, max_to_keep=3)

In [ ]:
@tf.function
def train_step(x, y):
    with tf.GradientTape() as tape:
        y_hat = ae(x)
        loss = l1_loss(y, y_hat)
    grads = tape.gradient(loss, ae.trainable_variables)
    opt.apply_gradients(zip(grads, ae.trainable_variables))
    return loss

In [ ]:
def log(loss, color, sketch, output, n=BATCH_SIZE):
    images = []
    for i in range(n):
        img_row = np.hstack([color[i], sketch[i], output[i]]) * 0.5 + 0.5
        images.append(img_row)
    images = wandb.Image(np.vstack(images), caption="Left: Color, Mid: Sketch, Right: Output")
    wandb.log({"loss": loss, "examples": images})

In [ ]:
def fit():
    val_iter = iter(val_dataset)
    for ep in range(EPOCHS):
        named_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
        print(f'\nEpoch: {ep + 1} of {EPOCHS}\nStarted: {time_string}')

        losses = []

        for idx, (colors, sketches) in tqdm(train_dataset.enumerate(), total=n_batches):
            loss = train_step(sketches, colors)
            losses.append(loss)

        loss = np.mean(np.array(losses))
        named_tuple = time.localtime()
        time_string = time.strftime("%m/%d/%Y, %H:%M:%S", named_tuple)
        print(f'\nEnded: {time_string}\nloss: {loss}')

        path = manager.save()
        print(f'\nCheckpoint saved in: {path}')

        colors, sketches = val_iter.next()
        outs = ae(sketches)
        log(loss, colors, sketches, outs, n=5)

In [ ]:
fit()

  0%|          | 0/508.0 [00:00<?, ?it/s]


Epoch: 1 of 20
Started: 06/18/2021, 15:04:18


100%|██████████| 508/508.0 [03:00<00:00,  2.82it/s]



Ended: 06/18/2021, 15:07:19
loss: 0.5684372186660767

Checkpoint saved in: /content/ckpt/ckpt-1


  0%|          | 0/508.0 [00:00<?, ?it/s]

...


Ended: 06/18/2021, 15:54:51
loss: 0.13152816891670227

Checkpoint saved in: /content/ckpt/ckpt-19


  0%|          | 0/508.0 [00:00<?, ?it/s]


Epoch: 20 of 20
Started: 06/18/2021, 15:54:51


100%|██████████| 508/508.0 [02:36<00:00,  3.25it/s]



Ended: 06/18/2021, 15:57:28
loss: 0.1298578530550003

Checkpoint saved in: /content/ckpt/ckpt-20


In [ ]:
ae.save(ckpt_dir + '/sketch_colorization_ae.h5')
model_artifact = wandb.Artifact("sketch_colorization_AE", type="model", 
                                description="autoencoder model checkpoint")
model_artifact.add_file(ckpt_dir + '/sketch_colorization_ae.h5')
run.log_artifact(model_artifact)